In [ ]:
from Ichigaya import chart
刹那旅行 = chart.chart(85, "expert")
刹那旅行.to_path("谱面")
json = 刹那旅行.load()

In [ ]:
chart_keys = {
    "Single": [], 
    "Hold": [], 
    "Flick": [], 
    "Hold&Flick": [], 
    "Left": [], 
    "Right": []
}
time_stamps = []
for i in json:
    if i["type"] == "BPM":
        time_stamps.append(i)
    elif i["type"] == "Single":
        if "flick" in i.keys():
            chart_keys["Flick"].append(i)
        else:
            chart_keys["Single"].append(i)
    elif i["type"] == "Directional":
        if i["direction"] == "Left":
            chart_keys["Left"].append(i)
        else:
            chart_keys["Right"].append(i)
    elif i["type"] != "System":
        if "flick" in i["connections"][-1].keys():
            chart_keys["Hold&Flick"].append(i)
        else:
            chart_keys["Hold"].append(i)


In [ ]:
chart_keys["Flick"]

In [ ]:
shadow = {}
for single in chart_keys["Single"]:
    beat = str(single["beat"])
    if beat in shadow.keys():
        shadow[beat].append(single["lane"])
    else:
        shadow.update({beat: [single["lane"]]})

In [ ]:
for hold in chart_keys["Hold"]:
    for node in [hold["connections"][0], hold["connections"][-1]]:
        beat = str(node["beat"])
        if beat in shadow.keys():
            shadow[beat].append(node["lane"])
        else:
            shadow.update({beat: [node["lane"]]})

In [ ]:
for hold in chart_keys["Hold&Flick"]:
    for node in [hold["connections"][0], hold["connections"][-1]]:
        beat = str(node["beat"])
        if beat in shadow.keys():
            shadow[beat].append(node["lane"])
        else:
            shadow.update({beat: [node["lane"]]})

In [ ]:
for single in chart_keys["Flick"]:
    beat = str(single["beat"])
    if beat in shadow.keys():
        shadow[beat].append(single["lane"])
    else:
        shadow.update({beat: [single["lane"]]})

for single in chart_keys["Left"]:
    beat = str(single["beat"])
    if beat in shadow.keys():
        shadow[beat].append(single["lane"])
    else:
        shadow.update({beat: [single["lane"]]})

for single in chart_keys["Right"]:
    beat = str(single["beat"])
    if beat in shadow.keys():
        shadow[beat].append(single["lane"])
    else:
        shadow.update({beat: [single["lane"]]})

In [ ]:
links = {}
for stamp in shadow.keys():
    if len(shadow[stamp]) > 1:
        links.update({stamp: shadow[stamp]})
print(links)

In [ ]:
key_shape = {
    "Single": " (===========) ", 
    "Flick": " (===^^^^^===) ", 
    "Hold_start": " /===========\\ ", 
    "Hold_end": " \\===========/ ", 
    "Hold_body": "  +         +  ", 
    "Hold_node": "  +   ---   +  ", 
    "Left": [
        " <===========0 ", 
        " <<<=======================0   ", 
        " <<<<<===================================0     "
    ], 
    "Right": [
        " 0===========> ", 
        "   0=======================>>> ", 
        "     0===================================>>>>> "
    ]
}
chart_pos = {
    "left": [1, 17, 33, 49, 65, 81, 97], 
    "right": [15, 31, 47, 63, 79, 95, 111],
    "middel": [8, 24, 40, 56, 72, 88, 104] 
}

In [ ]:
bpm =127
length = 288 + 1
freq = 60 / bpm
expand = 1 / freq
if expand < 1.41: bps = 1
elif expand < 2.82: bps = 2
else: bps = 4
lpb = int(64 / bps)
chart_line = lpb * length
blank = ("|" + " " * 15)*7 + "|"
chart_text = [blank] * chart_line

In [ ]:
for single in chart_keys["Single"]:
    line = int(float(single["beat"]) * lpb - 1)
    lane = single["lane"]
    node = chart_pos["left"][lane], chart_pos["right"][lane]
    line_chart = chart_text[line]
    line_chart = line_chart[:node[0]] + key_shape["Single"] + line_chart[node[1]+1:]
    chart_text[line] = line_chart

In [ ]:
for single in chart_keys["Flick"]:
    line = int(float(single["beat"]) * lpb - 1)
    lane = single["lane"]
    node = chart_pos["left"][lane], chart_pos["right"][lane]
    line_chart = chart_text[line]
    line_chart = line_chart[:node[0]] + key_shape["Flick"] + line_chart[node[1]+1:]
    chart_text[line] = line_chart

In [ ]:
for beat in links.keys():
    line = int(float(beat) * lpb - 1)
    if links[beat][0] < links[beat][0]:
        node = chart_pos["middel"][links[beat][0]], chart_pos["middel"][links[beat][1]]
    else: 
        node = chart_pos["middel"][links[beat][1]], chart_pos["middel"][links[beat][0]]
    line_chart = chart_text[line]
    new_line_chart = ""
    for i in range(len(line_chart)):
        if line_chart[i] in [" ", "|"] and (i >= node[0] and i <= node[1]):
            new_line_chart += "-"
        else:
            new_line_chart += line_chart[i]
    chart_text[line] = new_line_chart

In [ ]:
for i in range(chart_line):
    # if chart_text[-i-1] != blank:
    print(chart_text[-i-1])